# SVI (Stochastic Variational Inference)

pyro以SVI作为其通用的inference algorithm。

$$p_{\theta}(X, Z)=p_{\theta}(X|Z)p_{\theta}(Z)$$

其中$X$表示观测，$Z$表示latent random variables，$\theta$是parameters。

组成$p_{\theta}$的$p_i$需要服从下面3个条件：

1. 我们可以从$p_i$中进行采样；
2. 可以计算每个点上的log pdf $p_i$;
3. $p_i$相对于参数$\theta$是可微分的。

我们现在的目标是：

$$\theta_{max}=argmax_{\theta}{\log p_{\theta}(X)}=\int{dZp_{\theta}(X,Z)}$$

并且得到一个基于最优theta值的后验分布：

$$p_{\theta_{max}}(Z|X)=\frac{p_{\theta_{max}}(X,Z)}{\int{dZp_{\theta_{max}}(X,Z)}}$$

其实最大的问题都在于那个积分是难以解决的。VI提供了一种解决方法。

## guide

基本思想是借助一个变分分布$q_{\phi}(Z)$，这在pyro中称为guide。它是最优后验分布$p_{\theta_{max}}(Z|X)$的近似。

在pyro中，这个guide也被使用`pyro.sample`来编写成一个随机函数，也就是model。其不包含observed data，但必须和真实的model拥有相同的输入。同时$Z$需要在model和guide中拥有相同的名称。

## ELBO

简单的推断可以得到下面的等式：

$$\log{p_{\theta}(X)} = ELBO + KL(q_{\phi}(Z)||p_{\theta}(Z|X))$$

其中

$$ELBO = \mathbb{E}_{q_{\phi}(Z)}[\log{p_{\theta}(X,Z)}-\log{q_{\phi}(Z)}]$$

1. 我们首先固定theta、改变phi来最大化ELBO，则其实就是最小化q和后验之间的KL散度。最终，ELBO就是$\log{p_{\theta}(X)}$。
2. 然后我们再固定phi、改变theta来最大化ELBO，这是就是最大化$\log{p_{\theta}(X)}$。

上述过程交替进行，最后得到我们想要得到的结果。但实际上可能是同时改变phi和theta来最大化ELBO（VAE实际上就是这么做的），这样依然可能得到我们想要的值。

## Optimizer

从上面我们看到，每一步还需要一个梯度下降优化器来最大化ELBO。pyro wraps了一下pytorch中的optimizer，在`pyro.optim`中，可以用于我们的最优化。

调用的方式有两种：

第一种是比较通用的，将优化器的参数包装成字典，传入优化器类即可。

In [6]:
from pyro.optim import Adam

adam_params = {"lr": 0.005, "betas": (0.95, 0.999)}
optimizer = Adam(adam_params)

第二种方式是传入一个callble，传入两个参数（`module_name`和`param_name`），返回的是参数字典。其可以提供更加细致的操作，比如对于不同的参数，使用不同的学习率。

In [7]:
def per_param_callable(module_name, param_name):
    if param_name == "my_special_parameter":
        return {"lr": 0.010}
    else:
        return {"lr": 0.001}
optimizer = Adam(per_param_callable)

**本来都写完了，但没保存都删了，那就这样吧。...**